In [1]:
import numpy as np
import pandas as pd

from bikewheelcalc import BicycleWheel, Hub, Rim
import bikewheellib as bl
from doetools import TensionBucklingExpDOE
from doetools.wheel_library import create_wheel_from_lm

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
dyn_dir = '../data/doe/tension_buckling_dynamic'

up = 0.001e-3
doe = TensionBucklingExpDOE(out_dir=dyn_dir,
                            opts={'spk_paired': False,
                                  'spk_eltype': 'beam',
                                  'rim_perturb': [0., up, up, up, up],
                                  'sim_time': 1.0})

R = 0.300
GJ = 25.0
EI1 = 100.
EI2 = GJ / 0.2

wh = BicycleWheel()
wh.hub = Hub(diam1=0.05, width1=0.025, width2=0.025)
wh.rim = Rim(radius=R, area=100e-6,
             I11=GJ / 26.0e9, I22=EI2 / 26.0e9, I33=EI1 / 26.0e9, Iw=0.0 / 69.0e9,
             young_mod=69.0e9, shear_mod=26.0e9)

wh.lace_radial(n_spokes=36, diameter=1.8e-3, young_mod=200e9, offset=0.00)

doe.add_experiment(wh, opts={'jobname': 'Tc_dynamic_2'})

print('\nCreated {0:d} simulations'.format(len(doe.db)))

doe.write_input_files(N_batches=1)
doe.to_csv()


Created 1 simulations
